# Boundary 

The task is to combine current velocities from an oceanographic model without tides with tidal current.

* The oceanographic model data is vertically resolved and available in a vertical transect as a Dfs2 with daily timestep
* The tidal model is vertically integrated and available in a line transect as a Dfs1 with hourly timestep
* The spatial grid is identical

## Steps
1. Read data
2. Interpolate in time
3. Calculate $\mathbf{U}_{combined} = \mathbf{U}_{tide} + \mathbf{U}_{residual}$
4. Write to new dfs2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mikeio import Dfs2, Dfs1

dfs2 = Dfs2("../tests/testdata/uv_vertical_daily.dfs2")
ds_surge = dfs2.read()
ds_surge

<mikeio.Dataset>
Dimensions: (3, 5, 11)
Time: 2021-08-14 12:00:00 - 2021-08-16 12:00:00
Items:
  0:  eastward_sea_water_velocity <u velocity component> (meter per sec)
  1:  northward_sea_water_velocity <v velocity component> (meter per sec)

In [2]:
dfs1 = Dfs1("../tests/testdata/vu_tide_hourly.dfs1")
ds_tide = dfs1.read()
ds_tide

<mikeio.Dataset>
Dimensions: (721, 11)
Time: 2021-08-01 00:00:00 - 2021-08-31 00:00:00
Items:
  0:  Tidal current component (geographic North) <v velocity component> (meter per sec)
  1:  Tidal current component (geographic East) <u velocity component> (meter per sec)

### 1. Interpolate in time

In [3]:
ds_surge_h = ds_surge.interp_time(ds_tide.time)
ds_surge_h

<mikeio.Dataset>
Dimensions: (721, 5, 11)
Time: 2021-08-01 00:00:00 - 2021-08-31 00:00:00
Items:
  0:  eastward_sea_water_velocity <u velocity component> (meter per sec)
  1:  northward_sea_water_velocity <v velocity component> (meter per sec)

### 2. Combine

**Note that the naming and order is inconsistent between the two data sources!**

In [4]:
u_tide = ds_tide["Tidal current component (geographic East)"]
u_tide = np.expand_dims(u_tide, 1)
u_tide.shape

(721, 1, 11)

In [5]:
u_surge = ds_surge_h["eastward_sea_water_velocity"]
u_surge.shape

(721, 5, 11)

In [6]:
u_combined = u_tide + u_surge
u_combined.shape

(721, 5, 11)

In [7]:
v_tide = ds_tide["Tidal current component (geographic North)"]
v_tide = np.expand_dims(v_tide, 1)
v_surge = ds_surge_h["northward_sea_water_velocity"]
v_combined = v_tide + v_surge

### 3. Write to dfs2

In [8]:
from mikeio import Dataset
ds_combined = Dataset(data=[u_combined, v_combined],
                      time=ds_tide.time,
                      items=ds_surge.items)
ds_combined

<mikeio.Dataset>
Dimensions: (721, 5, 11)
Time: 2021-08-01 00:00:00 - 2021-08-31 00:00:00
Items:
  0:  eastward_sea_water_velocity <u velocity component> (meter per sec)
  1:  northward_sea_water_velocity <v velocity component> (meter per sec)

In [9]:
ds_combined = ds_combined.dropna()
ds_combined

<mikeio.Dataset>
Dimensions: (49, 5, 11)
Time: 2021-08-14 12:00:00 - 2021-08-16 12:00:00
Items:
  0:  eastward_sea_water_velocity <u velocity component> (meter per sec)
  1:  northward_sea_water_velocity <v velocity component> (meter per sec)

In [10]:
dfsnew = Dfs2()

dfsnew.write("uv_combined.dfs2", ds_combined)

c:\users\jan\code\mikeio\mikeio\dfs.py:255: UserWarning: No coordinate system provided
  warnings.warn("No coordinate system provided")


# Clean up


In [11]:
import os
os.remove("uv_combined.dfs2")

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'uv_combined.dfs2'